## Die komplexe Matrixexponentialfunktion $e^A$, ihre numerische Auswertung und ingenieurwissenschaftliche Anwendung

Zuerst importieren wir das Paket Sympy und verändern Einstellungen für die bessere Darstellung.

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from sympy import * # das Computeralgebrasystem SymPy wird importiert
init_printing() # schönere Darstellung

Wir betrachten ein System von zwei Gewichten und drei Federn.

![](Feder_2.svg)

Wir können dieses System durch folgende Differentialgleichung beschreiben:

\begin{equation*}
\ddot Y
=
\begin{pmatrix}
-2 \omega & \omega \\
\omega & -2 \omega
\end{pmatrix}
Y
\end{equation*}

Wir führen wieder die Variablen $p_1 := \dot y_1$ und $p_2 = \dot y_2$ ein.
Dann haben wir

\begin{equation*}
\dot P =
\begin{pmatrix}
-2 \omega & \omega \\
\omega & -2 \omega
\end{pmatrix}
Y
\text{ und }
\dot Y =
\begin{pmatrix}
1 & 0 \\
0 & 1
\end{pmatrix}
P.
\end{equation*}

Wir können unser Problem somit als

\begin{equation*}
\begin{pmatrix}
\dot p_1 \\ \dot p_2 \\ \dot y_1 \\ \dot y_2
\end{pmatrix}
=
\underbrace{
\begin{pmatrix}
-2 \omega & \omega & 0 & 0 \\
\omega & -2 \omega & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{pmatrix}
}_{:=A}
\begin{pmatrix}
y_1 \\ y_2 \\ p_1 \\ p_2
\end{pmatrix}
\end{equation*}
formulieren.


In [ ]:
A = Matrix([
    [0, 0, -2, 1],
    [0, 0, 1, -2],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

A

Wir bestimmen die Eigenwerte und Eigenvektoren von $A$:

In [ ]:
A.eigenvects()

Wir diagonalisieren $A$. Als Ergebnis wird die Matrix der Eigenvektoren, sowie die Diagonalmatrix ausgegeben.

In [ ]:
A.diagonalize()

Jetzt berechnen wir $\exp(tA)$. Da dies eine symbolische Rechnung ist, müssen wir das Symbol $t$ definieren.

In [ ]:
t = Symbol('t')

exp_tA = exp(t*A)

exp_tA

Mit dem Befehl ``Simplify`` wird das Ergebnis übersichtlicher:

In [ ]:
exp_tA = simplify(exp_tA)

exp_tA

Wir sehen aber, dass die Formel

\begin{equation*}
i \exp(-ix) - i \exp(ix) = 2 \sin(x)
\end{equation*}

noch nicht angewendet wurde.
Dies können wir auch noch durch die Funktion ``rewrite`` erzwingen:

In [ ]:
exp_tA = simplify(exp_tA.rewrite(sin))

exp_tA

Da $Y = (y_1, y_2, p_1, p_2)^\top$, und wir uns für den Fall interessieren, dass $\dot y_1(0) = 0$ und $\dot y_2(0) = 0$, is für uns die obere $2\times 2$-Matrix von $A$,

\begin{equation*}
\begin{pmatrix}
a_{11} & a_{12} \\
a_{21} & a_{22}
\end{pmatrix}
\end{equation*}

relevant.

In [ ]:
exp_tA[[0, 1], [0, 1]]

Unser Ergebnis ist somit:

In [ ]:
y_1 = Symbol('y_1')
y_2 = Symbol('y_2')

exp_tA[[0, 1], [0, 1]] * Matrix([y_1, y_2])

Jetzt betrachten wir den Fall, dass das erste Gewicht die Hälfte des Gewichtes vom zweiten Gewicht hat, $m_1 = \frac{1}{2} m_2$.
Wir können unser Problem somit als

\begin{equation*}
\begin{pmatrix}
\dot p_1 \\ \dot p_2 \\ \dot y_1 \\ \dot y_2
\end{pmatrix}
=
\underbrace{
\begin{pmatrix}
- \omega & \frac{1}{2} \omega & 0 & 0 \\
\omega & -2 \omega & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{pmatrix}
}_{:=A}
\begin{pmatrix}
y_1 \\ y_2 \\ p_1 \\ p_2
\end{pmatrix}
\end{equation*}

formulieren.

In [ ]:
A = Matrix([
    [0, 0, -4, 2],
    [0, 0, 1, -2],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

A

In [ ]:
exp_tA = exp(t*A)
exp_tA

## Numerische Berechnung des Matrixexponentials

Die symbolischen Ergebnisse werden hier deutlich zu kompliziert. Wir können das Matrixexponential aber auch numerisch berechnen.

In [ ]:
from scipy.linalg import expm
import numpy as np 

A = np.array([
    [0, 0, -1, 0.5],
    [0, 0, 1, -2],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

A

In [ ]:
expm(A)

## Kompartmentanalyse

![](Tanks_1.svg)

Wir können dieses Problem als

\begin{equation*}
\dot Y =
	\begin{pmatrix}
		-\frac{1}{2} & 0 & 0 \\
		\frac{1}{2} & -\frac{1}{4} & 0 \\
		0 & \frac{1}{4} & - \frac{1}{6}
	\end{pmatrix}
	Y
\end{equation*}

darstellen.

In [ ]:
B = Matrix([
    [-Rational(1, 2), 0, 0],
    [Rational(1, 2), -Rational(1, 4), 0],
    [0, Rational(1, 4), -Rational(1, 6)]
])

B

In [ ]:
B.diagonalize()

In [ ]:
simplify(exp(t * B))

Unsere Lösung ist schließlich:

In [ ]:
simplify(exp(t * B)) * Matrix([2, 4, 6])

![](Kaskade_Ergebnis.svg)

## Berechnung der Jordan-Normalform in Scipy

In [ ]:
C = Matrix([[1, 0, 1], [0, 1, 1], [0, 0, 1]])

C.jordan_form()

In [ ]:
M = Matrix([
    [5, 4, 2, 1],
    [0, 1, -1, -1],
    [-1, -1, 3, 0],
    [1, 1, -1, 2]
])

M

In [ ]:
M.jordan_form()